In [1]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
import pickle
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
def read_database(db):
    connect_string = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8mb4'.format("admin", "SoftwarePass!", 'database-1.chkvd5qjrkby.us-east-1.rds.amazonaws.com', 3306, "dublinbikes")
    engine = create_engine(connect_string, echo=False)
    df = pd.read_sql(db, engine)
    return df

Bikes = read_database("Bikes")#or "Weather", "Bikes"

In [2]:
Bikes

,name,available_bike_stands,available_bikes,status,last_update
0,SMITHFIELD NORTH,24,6,OPEN,25 Feb 2020 09:47:33
1,PARNELL SQUARE NORTH,18,2,OPEN,25 Feb 2020 09:45:01
2,CLONMEL STREET,23,9,OPEN,25 Feb 2020 09:40:11
3,AVONDALE ROAD,34,6,OPEN,25 Feb 2020 09:48:40
4,MOUNT STREET LOWER,1,38,OPEN,25 Feb 2020 09:49:48
...,...,...,...,...,...
1310371,WILTON TERRACE,15,5,OPEN,07 Apr 2020 09:34:05
1310372,EMMET ROAD,20,19,OPEN,07 Apr 2020 09:40:42
1310373,HEUSTON BRIDGE (NORTH),20,19,OPEN,07 Apr 2020 09:40:45
1310374,LEINSTER STREET SOUTH,15,15,OPEN,07 Apr 2020 09:42:12


In [3]:
Weather = read_database("Weather")#or "Weather", "Bikes"

In [4]:
Weather.nunique()

current_weather        6
detailed_weather      17
temperature           18
last_update         7660
dtype: int64

In [5]:
Weather['current_weather'].value_counts()

Clouds     9765
Rain       1804
Drizzle     248
Mist        112
Clear        88
Fog          53
Name: current_weather, dtype: int64

In [6]:
Weather = Weather[['current_weather', 'temperature', 'last_update']]
Weather

,current_weather,temperature,last_update
0,Clouds,3,25 Feb 2020 09:43:00
1,Clouds,3,25 Feb 2020 09:50:02
2,Clouds,4,25 Feb 2020 10:00:02
3,Clouds,4,25 Feb 2020 10:05:03
4,Clouds,4,25 Feb 2020 10:05:03
...,...,...,...
12065,Clouds,10,07 Apr 2020 09:21:15
12066,Clouds,10,07 Apr 2020 09:21:15
12067,Clouds,11,07 Apr 2020 09:35:04
12068,Clouds,11,07 Apr 2020 09:35:04


In [7]:
Weather['last_update'] = pd.to_datetime(Weather['last_update'])
Weather

,current_weather,temperature,last_update
0,Clouds,3,2020-02-25 09:43:00
1,Clouds,3,2020-02-25 09:50:02
2,Clouds,4,2020-02-25 10:00:02
3,Clouds,4,2020-02-25 10:05:03
4,Clouds,4,2020-02-25 10:05:03
...,...,...,...
12065,Clouds,10,2020-04-07 09:21:15
12066,Clouds,10,2020-04-07 09:21:15
12067,Clouds,11,2020-04-07 09:35:04
12068,Clouds,11,2020-04-07 09:35:04


In [8]:
Weather2 = pd.get_dummies(Weather['current_weather'])

In [9]:
Weather3 = pd.concat([Weather, Weather2], axis=1)
Weather3

,current_weather,temperature,last_update,Clear,Clouds,Drizzle,Fog,Mist,Rain
0,Clouds,3,2020-02-25 09:43:00,0,1,0,0,0,0
1,Clouds,3,2020-02-25 09:50:02,0,1,0,0,0,0
2,Clouds,4,2020-02-25 10:00:02,0,1,0,0,0,0
3,Clouds,4,2020-02-25 10:05:03,0,1,0,0,0,0
4,Clouds,4,2020-02-25 10:05:03,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
12065,Clouds,10,2020-04-07 09:21:15,0,1,0,0,0,0
12066,Clouds,10,2020-04-07 09:21:15,0,1,0,0,0,0
12067,Clouds,11,2020-04-07 09:35:04,0,1,0,0,0,0
12068,Clouds,11,2020-04-07 09:35:04,0,1,0,0,0,0


In [10]:
Weather3 = Weather3.set_index('last_update')

In [11]:
# Weather = Weather.groupby('last_update')['current_weather', 'temperature'].mean().reset_index()

# Weather = Weather.resample('H').mean()
Weather3 = Weather3.resample('15min').mean()

In [12]:
for data in ['Clear', 'Clouds', 'Drizzle', 'Fog', 'Mist', 'Rain']:
    Weather3[data] = Weather3[data].apply(lambda x: 0 if x < 0.5 else 1)

In [13]:
Weather3

,temperature,Clear,Clouds,Drizzle,Fog,Mist,Rain
last_update,,,,,,,
2020-02-25 09:30:00,3.0,0,1,0,0,0,0
2020-02-25 09:45:00,3.0,0,1,0,0,0,0
2020-02-25 10:00:00,4.0,0,1,0,0,0,0
2020-02-25 10:15:00,4.0,0,1,0,0,0,0
2020-02-25 10:30:00,4.0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
2020-04-07 08:30:00,9.8,0,1,0,0,0,0
2020-04-07 08:45:00,10.0,0,1,0,0,0,0
2020-04-07 09:00:00,10.0,0,1,0,0,0,0


In [14]:
names = Bikes.name.unique()
def prepro(name):
    return name.replace('/', 'slash')

In [15]:
for name in names:
    Bikes2 = Bikes.loc[Bikes['name'] == name]#isolate one station

    Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
    Bikes2 = Bikes2.set_index('last_update')
    # Bikes = Bikes.groupby('Station').resample('30min').mean().reset_index('Station')
    Bikes2 = Bikes2.resample('15min').mean()
    Bikes2 = Bikes2.round()


    df = pd.concat([Bikes2, Weather3], axis=1)
    df['last_update'] = df.index

    df = df.reset_index(drop=True)
    

    df.dropna(subset = ["available_bike_stands"], inplace=True)
    

    df['Last_updated_day'] = df['last_update'].dt.day
    df['Last_updated_hour'] = df['last_update'].dt.hour
    df['Last_updated_minute'] = df['last_update'].dt.minute
    df['Last_updated_dayofweek'] = df['last_update'].dt.dayofweek
    

    df.dropna(subset = ["temperature"], inplace=True)

    y = df.available_bikes
    # Create X
    features = ['temperature', 'Clear', 'Clouds', 'Drizzle', 'Fog', 'Mist', 'Rain', 'Last_updated_day', 'Last_updated_hour', 'Last_updated_minute','Last_updated_dayofweek']
    X = df[features]


    # Split into validation and training data
    train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

    # Specify Model
    # Previous version had MAE of 2.4 this one had 0.98
    rf_model = RandomForestRegressor(n_estimators=100, max_features='auto', oob_score=True, random_state=1)
    # Fit Model
    rf_model.fit(train_X, train_y)

    # # Make validation predictions and calculate mean absolute error
    # val_predictions = rf_model.predict(val_X)
    # val_mae = mean_absolute_error(val_predictions, val_y)
    # print("Validation MAE max_leaf_nodes = 50: {:,.0f}".format(val_mae))


    filename = f'models/{prepro(name)}.sav'
    pickle.dump(rf_model, open(filename, 'wb'))

<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bikes2['last_update'] = pd.to_datetime(Bikes2['last_update'])
<ipython-input-15-bb68f6a31303>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http